# PyHEP WG topical meeting - boost-histogram / Hist

### Henry Schreiner (Princeton University) and Aman Goel (University of Delhi)

**March 2nd, 2022**

In [ ]:
import hist
from hist import Hist

Run the code with us through Binder, altering examples and asking "what if" questions along the way :)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/henryiii/histogram-tutorial/master?filepath=)

In [ ]:
## outline (to be removed)

# defining a histogram and different ways
# manipulation of the histogram
# features and examples

In [ ]:
import numpy as np

Let's prepare a default random number generator (numpy 1.17+)

In [ ]:
rng = np.random.default_rng()

And let's make something interesting to histogram:

In [ ]:
data = np.hstack(
    [
        rng.normal(0, 2, size=40_000),
        rng.normal(3, 0.3, size=5_000),
        rng.normal(-3, 0.3, size=5_000),
    ]
)

Now, let's make a histogram:

In [ ]:
h = Hist.new.Reg(100, -10, 10).Int64().fill(data)
h

In [ ]:
h.plot();

In [ ]:
h[-4j:4j].plot();

In [ ]:
# NumPy API
# hist.numpy.histogram(data, bins=100, histogram=Hist)

## What is a Hist object made up of?

- Axis objects
- Storage

## Defining a histogram with Hist
Let's first import Hist and quickly define a histogram!

### Classic definition

In [ ]:
h = Hist(
    hist.axis.Regular(10, 0, 1, name="x"),
    hist.axis.Regular(10, 0, 1, name="y"),
    storage=hist.storage.Weight(),
)
print(h)

### QuickConstruct

In [ ]:
h = Hist.new.Reg(10, 0, 1, name="x").Reg(10, 0, 1, name="y").Weight()
print(h)

In [ ]:
## definitions

# regular and quick-construct
# fill

## Playing with the Hist Object

In [ ]:
# slicing & rebinning
# accessing various properties
# setting content

## Diving into the Axes

In [ ]:
# types and manipulation
# include transform here

## Quick Overview of Storages

In [ ]:
# skip Accumulators?

## Repr Examples

In [ ]:
# show quick examples of previous objects as Reprs or combine this section above

## Plots and Histoprint

In [ ]:
# show various plots, maybe go through different levels to build one example

## Stacks

In [ ]:
# look for an interesting example?